In [26]:
pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 36.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from PIL import Image
import os

# Define paths
original_images_folder = "/Users/atharva/Desktop/Master_data"
annotations_folder = "/Users/atharva/Desktop/Master_data"
output_folder = "/Users/atharva/Desktop/Patched_master_data"

# Define patch size
patch_size = 160

# Get a list of image files
image_files = [f for f in os.listdir(original_images_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Create output folder if not exists
os.makedirs(output_folder, exist_ok=True)

# Loop through each image file
for image_file in image_files:
    image_path = os.path.join(original_images_folder, image_file)
    annotation_path = os.path.join(annotations_folder, os.path.splitext(image_file)[0] + ".txt")

    # Load the original image
    original_image = Image.open(image_path)
    image_width, image_height = original_image.size

    # Load annotations from the YOLO format
    annotations = []
    with open(annotation_path, 'r') as f:
        for line in f:
            annotations.append(list(map(float, line.strip().split())))

    num_patches_x = image_width // patch_size
    num_patches_y = image_height // patch_size


# Loop through each patch
    for y in range(num_patches_y):
        for x in range(num_patches_x):
            left = x * patch_size
            upper = y * patch_size
            right = left + patch_size
            lower = upper + patch_size

            # Crop the patch
            patch = original_image.crop((left, upper, right, lower))

            # Save the patch
            patch_filename = f"{os.path.splitext(image_file)[0]}_patch_{x}_{y}.jpg"
            patch.save(os.path.join(output_folder, patch_filename))

            # Convert annotations for the patch
            patch_annotations = []
            for annotation in annotations:
                ann_class, ann_x, ann_y, ann_w, ann_h = annotation

                # Calculate annotation's boundaries
                ann_left = ann_x - ann_w / 2
                ann_right = ann_x + ann_w / 2
                ann_upper = ann_y + ann_h / 2
                ann_lower = ann_y - ann_h / 2

                # Check if annotation is within the boundaries of the patch
                if (left/640 <= ann_x <= right/640 and upper/640 <= ann_y <= lower/640) and \
                   (left/640 <= ann_left <= right/640 and upper/640 <= ann_upper <= lower/640) and \
                   (left/640 <= ann_right <= right/640 and upper/640 <= ann_lower <= lower/640):
                    ann_x_rel = (ann_x - left/640)*4
                    ann_y_rel = (ann_y - upper/640)*4
                    ann_w_rel = ann_w *4
                    ann_h_rel = ann_h *4
                    patch_annotations.append(f"{0} {ann_x_rel:.6f} {ann_y_rel:.6f} {ann_w_rel:.6f} {ann_h_rel:.6f}")
#             print("-------------------")
#             print(patch_annotations)
#             print("-------------------")

            #Save annotations for the patch if there are any
            if patch_annotations:
                with open(os.path.join(output_folder, f"{os.path.splitext(image_file)[0]}_patch_{x}_{y}_.txt"), 'w') as patch_ann_file:
                    patch_ann_file.write('\n'.join(patch_annotations))


In [2]:
import cv2
import numpy as np
import os

def parse_yolo_annotation(annotation_path):
    with open(annotation_path, 'r') as file:
        lines = file.read().splitlines()
    
    annotations = []
    for line in lines:
        values = line.split()
        class_id = int(values[0])
        x_center, y_center, width, height = map(float, values[1:])
        annotations.append((class_id, x_center, y_center, width, height))
    
    return annotations
def draw_boxes_on_image(image_path, annotations, class_names):
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    for annotation in annotations:
        class_id, x_center, y_center, bbox_width, bbox_height = annotation
        left = int((x_center - bbox_width / 2) * width)
        top = int((y_center - bbox_height / 2) * height)
        right = int((x_center + bbox_width / 2) * width)
        bottom = int((y_center + bbox_height / 2) * height)
        
#         label = class_names[class_id]
        
        cv2.rectangle(image, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(image, '.', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return image


In [ ]:
annotation_path = '/Users/atharva/Desktop/Patched_master_data/consep_10_patch_1_2_.txt'
image_path = '/Users/atharva/Desktop/Patched_master_data/consep_10_patch_1_2.jpg'

annotations = parse_yolo_annotation(annotation_path)
class_names = ['']
annotated_image = draw_boxes_on_image(image_path, annotations, class_names)

cv2.imshow('Annotated Image', annotated_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
